In [1]:
import pandas as pd

# parquet 파일 경로 지정
file_path = 'logic_ded.parquet'

# pyarrow(또는 fastparquet) 기반으로 parquet 파일 읽기
df = pd.read_parquet(file_path)

In [2]:
df.head()

,cleaning_status,platinum_prompt,platinum_prompt_no_cot,platinum_target,original_target,platinum_parsing_strategy,input,target
0,consensus,Answer the following question. \n\nThe followi...,Answer the following question. \n\nThe followi...,[(a)],[(a)],bbh_multiple_choice,The following paragraphs each describe a set o...,(A)
1,consensus,Answer the following question. \n\nThe followi...,Answer the following question. \n\nThe followi...,[(b)],[(b)],bbh_multiple_choice,The following paragraphs each describe a set o...,(B)
2,verified,Answer the following question. \n\nThe followi...,Answer the following question. \n\nThe followi...,[(c)],[(c)],bbh_multiple_choice,The following paragraphs each describe a set o...,(C)
3,consensus,Answer the following question. \n\nThe followi...,Answer the following question. \n\nThe followi...,[(b)],[(b)],bbh_multiple_choice,The following paragraphs each describe a set o...,(B)
4,consensus,Answer the following question. \n\nThe followi...,Answer the following question. \n\nThe followi...,[(b)],[(b)],bbh_multiple_choice,The following paragraphs each describe a set o...,(B)


In [7]:
df['platinum_prompt'][0]

'Answer the following question. \n\nThe following paragraphs each describe a set of three objects arranged in a fixed order. The statements are logically consistent within each paragraph. On a branch, there are three birds: a blue jay, a quail, and a falcon. The falcon is to the right of the blue jay. The blue jay is to the right of the quail.\nOptions:\n(A) The blue jay is the second from the left\n(B) The quail is the second from the left\n(C) The falcon is the second from the left\n\nThink step-by-step. Then, provide the final answer in the format: "Answer: (A)".'

In [8]:
import pandas as pd
import re

# 1. Read the question again: ...
def repeat_question(match):
    # match.group(1): 첫 번째 줄바꿈 (예: "\n\n" 또는 "\n   \n")
    # match.group(2): 질문 텍스트
    # match.group(3): 두 번째 줄바꿈 (예: "\n\n")
    # 원하는 결과: 첫 번째 줄바꿈 뒤 기존 질문 대신 "Read the question again:" + 질문 텍스트를 두 번 반복하고,
    # 두 번째 줄바꿈을 그대로 붙인다.
    return f"{match.group(1)}{match.group(2)}\n\nRead the question again: {match.group(2)}{match.group(3)}"

# \n\n 사이에 공백이 있을 수도 있으므로 \n\s*\n 형태로 패턴을 수정
df['RE2'] = df['platinum_prompt'].str.replace(
    r'(\n\s*\n)(.*?)(\n\s*\n)(?=Think step-by-step)',
    repeat_question,
    flags=re.DOTALL,
    regex=True
)

# 2. Summarize the question first.
df['sum'] = df['platinum_prompt'].str.replace(
    r'(?=Think step-by-step)',
    "Summarize the question first. ",
    regex=True
)

# 3. Organize the question into a table first.
df['table'] = df['platinum_prompt'].str.replace(
    r'(?=Think step-by-step)',
    "Organize the question into a table first. ",
    regex=True
)

# 4. Organize the question into a graph structure first.
df['graph'] = df['platinum_prompt'].str.replace(
    r'(?=Think step-by-step)',
    "Organize the question into a graph structure first. ",
    regex=True
)

# 5. Summarize the question in bullet points first.
df['bullet_point'] = df['platinum_prompt'].str.replace(
    r'(?=Think step-by-step)',
    "Summarize the question in bullet points first. ",
    regex=True
)

# 6. Read the question again first.
df['sRE2'] = df['platinum_prompt'].str.replace(
    r'(?=Think step-by-step)',
    "Read the question again first. ",
    regex=True
)

In [13]:
df['sum'][0]

'Answer the following question. \n\nThe following paragraphs each describe a set of three objects arranged in a fixed order. The statements are logically consistent within each paragraph. On a branch, there are three birds: a blue jay, a quail, and a falcon. The falcon is to the right of the blue jay. The blue jay is to the right of the quail.\nOptions:\n(A) The blue jay is the second from the left\n(B) The quail is the second from the left\n(C) The falcon is the second from the left\n\nSummarize the question first. Think step-by-step. Then, provide the final answer in the format: "Answer: (A)".'

In [16]:
df['platinum_prompt_no_cot'][0]

'Answer the following question. \n\nThe following paragraphs each describe a set of three objects arranged in a fixed order. The statements are logically consistent within each paragraph. On a branch, there are three birds: a blue jay, a quail, and a falcon. The falcon is to the right of the blue jay. The blue jay is to the right of the quail.\nOptions:\n(A) The blue jay is the second from the left\n(B) The quail is the second from the left\n(C) The falcon is the second from the left\n\nThen, provide the final answer in the format: "Answer: (A)".'

In [17]:
import pandas as pd
import re

def repeat_question(match):
    # match.group(1): 첫 번째 줄바꿈 (예: "\n\n" 또는 "\n   \n")
    # match.group(2): 질문 텍스트
    # match.group(3): 두 번째 줄바꿈 (예: "\n\n")
    # 원하는 결과: 첫 번째 줄바꿈 뒤 기존 질문 대신 "Read the question again:" + 질문 텍스트를 두 번 반복하고,
    # 두 번째 줄바꿈을 그대로 붙인다.
    return f"{match.group(1)}{match.group(2)}\n\nRead the question again: {match.group(2)}{match.group(3)}"

# \n\n 사이에 공백이 있을 수도 있으므로 \n\s*\n 형태로 패턴을 수정
df['RE2_no_cot'] = df['platinum_prompt_no_cot'].str.replace(
    r'(\n\s*\n)(.*?)(\n\s*\n)(?=Then, provide the final answer in the format)',
    repeat_question,
    flags=re.DOTALL,
    regex=True
)

df['sum_no_cot'] = df['platinum_prompt_no_cot'].str.replace(
    r'(?=Then, provide the final answer in the format)',
    "Summarize the question first. ",
    regex=True
)

df['table_no_cot'] = df['platinum_prompt_no_cot'].str.replace(
    r'(?=Then, provide the final answer in the format)',
    "Organize the question into a table first. ",
    regex=True
)

df['graph_no_cot'] = df['platinum_prompt_no_cot'].str.replace(
    r'(?=Then, provide the final answer in the format)',
    "Organize the question into a graph structure first. ",
    regex=True
)

df['bullet_point_no_cot'] = df['platinum_prompt_no_cot'].str.replace(
    r'(?=Then, provide the final answer in the format)',
    "Summarize the question in bullet points first. ",
    regex=True
)

df['sRE2_no_cot'] = df['platinum_prompt_no_cot'].str.replace(
    r'(?=Then, provide the final answer in the format)',
    "Read the question again first. ",
    regex=True
)

In [23]:
df['RE2_no_cot'][0]

'Answer the following question. \n\nThe following paragraphs each describe a set of three objects arranged in a fixed order. The statements are logically consistent within each paragraph. On a branch, there are three birds: a blue jay, a quail, and a falcon. The falcon is to the right of the blue jay. The blue jay is to the right of the quail.\nOptions:\n(A) The blue jay is the second from the left\n(B) The quail is the second from the left\n(C) The falcon is the second from the left\n\nRead the question again: The following paragraphs each describe a set of three objects arranged in a fixed order. The statements are logically consistent within each paragraph. On a branch, there are three birds: a blue jay, a quail, and a falcon. The falcon is to the right of the blue jay. The blue jay is to the right of the quail.\nOptions:\n(A) The blue jay is the second from the left\n(B) The quail is the second from the left\n(C) The falcon is the second from the left\n\nThen, provide the final ans

In [24]:
df.to_parquet('logic_ded_add_task.parquet')

In [25]:
df.head()

,cleaning_status,platinum_prompt,platinum_prompt_no_cot,platinum_target,original_target,platinum_parsing_strategy,input,target,RE2,sum,table,graph,bullet_point,sRE2,RE2_no_cot,sum_no_cot,table_no_cot,graph_no_cot,bullet_point_no_cot,sRE2_no_cot
0,consensus,Answer the following question. \n\nThe followi...,Answer the following question. \n\nThe followi...,[(a)],[(a)],bbh_multiple_choice,The following paragraphs each describe a set o...,(A),Answer the following question. \n\nThe followi...,Answer the following question. \n\nThe followi...,Answer the following question. \n\nThe followi...,Answer the following question. \n\nThe followi...,Answer the following question. \n\nThe followi...,Answer the following question. \n\nThe followi...,Answer the following question. \n\nThe followi...,Answer the following question. \n\nThe followi...,Answer the following question. \n\nThe followi...,Answer the following question. \n\nThe followi...,Answer the following question. \n\nThe followi...,Answer the following question. \n\nThe followi...
1,consensus,Answer the following question. \n\nThe followi...,Answer the following question. \n\nThe followi...,[(b)],[(b)],bbh_multiple_choice,The following paragraphs each describe a set o...,(B),Answer the following question. \n\nThe followi...,Answer the following question. \n\nThe followi...,Answer the following question. \n\nThe followi...,Answer the following question. \n\nThe followi...,Answer the following question. \n\nThe followi...,Answer the following question. \n\nThe followi...,Answer the following question. \n\nThe followi...,Answer the following question. \n\nThe followi...,Answer the following question. \n\nThe followi...,Answer the following question. \n\nThe followi...,Answer the following question. \n\nThe followi...,Answer the following question. \n\nThe followi...
2,verified,Answer the following question. \n\nThe followi...,Answer the following question. \n\nThe followi...,[(c)],[(c)],bbh_multiple_choice,The following paragraphs each describe a set o...,(C),Answer the following question. \n\nThe followi...,Answer the following question. \n\nThe followi...,Answer the following question. \n\nThe followi...,Answer the following question. \n\nThe followi...,Answer the following question. \n\nThe followi...,Answer the following question. \n\nThe followi...,Answer the following question. \n\nThe followi...,Answer the following question. \n\nThe followi...,Answer the following question. \n\nThe followi...,Answer the following question. \n\nThe followi...,Answer the following question. \n\nThe followi...,Answer the following question. \n\nThe followi...
3,consensus,Answer the following question. \n\nThe followi...,Answer the following question. \n\nThe followi...,[(b)],[(b)],bbh_multiple_choice,The following paragraphs each describe a set o...,(B),Answer the following question. \n\nThe followi...,Answer the following question. \n\nThe followi...,Answer the following question. \n\nThe followi...,Answer the following question. \n\nThe followi...,Answer the following question. \n\nThe followi...,Answer the following question. \n\nThe followi...,Answer the following question. \n\nThe followi...,Answer the following question. \n\nThe followi...,Answer the following question. \n\nThe followi...,Answer the following question. \n\nThe followi...,Answer the following question. \n\nThe followi...,Answer the following question. \n\nThe followi...
4,consensus,Answer the following question. \n\nThe followi...,Answer the following question. \n\nThe followi...,[(b)],[(b)],bbh_multiple_choice,The following paragraphs each describe a set o...,(B),Answer the following question. \n\nThe followi...,Answer the following question. \n\nThe followi...,Answer the following question. \n\nThe followi...,Answer the following question. \n\nThe followi...,Answer the following question. \n\nThe followi...,Answer the following question. \n\nThe followi...,Answer the following question. \n\nThe followi...,Answer the follo